In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf

In [3]:
def get_balancesheet(ticker):
    stock_obj = yf.Ticker(ticker)
    return stock_obj.balancesheet

In [4]:
def income_statement(ticker):
    stock_obj = yf.Ticker(ticker)
    return stock_obj.financials

In [5]:
def get_info(ticker):
    df = {}
    ticker_info = yf.Ticker(ticker).info
    df['regularMarketPrice'] = ticker_info['regularMarketPrice']
    df['trailingEps'] = ticker_info['trailingEps']
    df['trailingAnnualDividendRate'] = ticker_info['trailingAnnualDividendRate']
    df['enterpriseValue'] = ticker_info['enterpriseValue']
    return df

In [21]:
def magic_formula(tickers):
    df = {}
    for ticker in tickers:
        print(ticker)
        balance_sheet = get_balancesheet(ticker).transpose()
        curr_assets = balance_sheet['Total Current Assets'].iloc[0]
        curr_liab = balance_sheet['Total Current Liabilities'].iloc[0]
        tang_assets = balance_sheet['Net Tangible Assets'].iloc[0]
        working_cap = curr_assets - curr_liab

        income_sheet = income_statement(ticker).transpose()

        EBIT = income_sheet['Ebit'].iloc[0]

        stats = get_info(ticker)
        if stats['trailingEps'] != None and stats['regularMarketPrice'] != None and stats['enterpriseValue'] != None:
            ROIC = EBIT / (working_cap + tang_assets)
            EY_perc = (stats['trailingEps'] / stats['regularMarketPrice']) * 100
            EY = EBIT / stats['enterpriseValue']
            DivdendRate = stats['trailingAnnualDividendRate']
            df[ticker] = [EY_perc, EY, ROIC, DivdendRate, EBIT]
    
    df = pd.DataFrame(df).transpose()
    df.columns = ["EarningsYield%", "EarningsYield", "ReturnOnInvestedCapital", 'trailingAnnualDividendRate', "EBIT"]
    df["CombineRanking"] = df["EarningsYield"].rank(ascending=False,na_option='bottom') + df["ReturnOnInvestedCapital"].rank(ascending=False,na_option='bottom')
    df["Magic"] = df["CombineRanking"].rank(method='first')
    return df.sort_values("Magic")

In [22]:
tickers = pd.read_csv('s_&_p_500_tickers.csv')['Ticker']
data = magic_formula(tickers)

MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
AOS
APA
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BK
BAX
BDX
BRK.B


KeyError: 'Total Current Assets'

In [20]:
data

,EarningsYield%,EarningsYield,ReturnOnInvestedCapital,trailingAnnualDividendRate,EBIT,CombineRanking,Magic
AAPL,3.431811,0.030333,0.639475,0.82,6.628800e+10,2.0,1.0
